In [2]:
from fyers_apiv3 import fyersModel
import webbrowser

redirect_uri= "https://127.0.0.1/"  
client_id = "CV26HLO9JI-100"                      
secret_key = "DST9UGHTX7"                           
grant_type = "authorization_code"                  
response_type = "code"                            
state = "sample"                              


appSession = fyersModel.SessionModel(client_id = client_id, redirect_uri = redirect_uri,response_type=response_type,state=state,secret_key=secret_key,grant_type=grant_type)

generateTokenUrl = appSession.generate_authcode()
print((generateTokenUrl))  
webbrowser.open(generateTokenUrl,new=1)


https://api-t1.fyers.in/api/v3/generate-authcode?client_id=CV26HLO9JI-100&redirect_uri=https%3A%2F%2F127.0.0.1%2F&response_type=code&state=sample


True

In [1]:
from fyers_apiv3 import fyersModel
from config import CLIENT_ID, AUTH_CODE, REDIRECT_URI, RESPONSE_TYPE, STATE, SECRET_KEY, GRANT_TYPE

# --- Exchange auth_code for access_token ---
session = fyersModel.SessionModel(
    client_id=CLIENT_ID,
    secret_key=SECRET_KEY,
    redirect_uri=REDIRECT_URI,
    response_type=RESPONSE_TYPE,
    state=STATE,
    grant_type=GRANT_TYPE
)

session.set_token(AUTH_CODE)
response = session.generate_token()

access_token = response.get("access_token")
print(access_token)

eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCcEVzVXZlbUlSaG5NS2pfVVFXcHU4cUdFVmk1TWI1WmpPMEhQT2xRRFZESHdNRExHc3RkU3oxSmQ3dDdMdWF6dm93WmVzTzNNODc2bzhOWUI0dXBmRUVZSGVjeC1GZDFWbThMQzJiZjRDZ2JTTUNNUT0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIzYjZlMjdjZmNjOTMxNGRlNGQwYWE5YWJhZWFkYjI0NjU5OTI2YTA5MjEyOTliYmNiOGMwNjRlYyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiRkFDNjIzNDQiLCJhcHBUeXBlIjoxMDAsImV4cCI6MTc2MjkwNzQwMCwiaWF0IjoxNzYyODM3ODA3LCJpc3MiOiJhcGkuZnllcnMuaW4iLCJuYmYiOjE3NjI4Mzc4MDcsInN1YiI6ImFjY2Vzc190b2tlbiJ9.uv_kzmdw0PUHQvJ6-FIy1zTd7a4u-l0INtVRxjcERkA


In [2]:
import pandas as pd
import numpy as np

import langchain
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
fyers = fyersModel.FyersModel(client_id=CLIENT_ID, token=access_token, is_async=False)

# data = {"symbols": "NSE:NIFTY50-INDEX"}
data = {"symbols": "NSE:TCS-EQ"}
response = fyers.quotes(data)
# print(response)

if "d" in response and len(response["d"]) > 0:
    ltp = response["d"][0]["v"]["lp"]  # 'lp' means Last Price
    print(f"Current price of {data['symbols']}: ₹{ltp}")
else:
    print("Error fetching price:", response)


Current price of NSE:TCS-EQ: ₹3006.7


## Equity

In [ ]:
import os
import re
import uuid  # For generating unique thread IDs
import sqlite3 # For listing existing threads
from dotenv import load_dotenv

# --- Core LangChain/Groq Imports ---
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool

# --- New Imports for Memory ---
from langchain.memory import ConversationSummaryBufferMemory
# We DON'T need 'from langchain.chat_models import ChatGroq', as it's wrong.
# The ChatGroq import from langchain_groq (line 9) is all we need.
from langchain_community.chat_message_histories import SQLChatMessageHistory # Backend for memory

# --- Fyers API ---
from fyers_apiv3 import fyersModel

# -----------------------------
# Configuration
# -----------------------------
DB_FILE = "chat_history.db" # File to store chat history
SQL_CONNECTION_STRING = f"sqlite:///{DB_FILE}"

# As requested: Manually added user info for customization
USER_PROFILE = """
- User Name: [Not Set]
- Interests: FinTech, Reinforcement Learning, AI-powered trading
- Preferred Response: Factual, concise, and provide strategies.
"""

# -----------------------------
# Load environment variables
# -----------------------------
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
# CLIENT_ID = os.getenv("FYERS_CLIENT_ID")
# access_token = os.getenv("FYERS_TOKEN")

if not GROQ_API_KEY or not CLIENT_ID or not access_token:
    print("Error: Missing environment variables.")
    print("Please set GROQ_API_KEY, FYERS_CLIENT_ID, and FYERS_TOKEN in your .env file.")
    exit()

print("Environment variables loaded.")

# -----------------------------
# Load Vector Store (for company search)
# -----------------------------
try:
    embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
    print("Company symbol vector store loaded!")
except Exception as e:
    print(f"Error loading vector store 'faiss_index': {e}")
    exit()

# -----------------------------
# Initialize LLM
# -----------------------------
# This single llm instance will be used for both the agent and the summarizer
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model_name="openai/gpt-oss-20b",
    temperature=0.1
)
print("LLM Initialized (llama3-70b-8192).")

# -----------------------------
# Initialize Fyers model
# -----------------------------
try:
    fyers = fyersModel.FyersModel(client_id=CLIENT_ID, token=access_token, is_async=False)
    test_response = fyers.get_profile()
    if test_response.get('s') != 'ok':
        print(f"Fyers API Error: {test_response.get('message')}")
        exit()
    print("Fyers connection successful.")
except Exception as e:
    print(f"Error initializing Fyers model: {e}")
    exit()

# -----------------------------
# Define Tools
# -----------------------------

@tool
def search_for_company_matches(company_query: str, top_k: int = 3) -> list[str]:
    """
    Searches the vector database for the top_k (default 3) most similar
    company names for a given query.
    Returns a list of raw strings, where each string contains the
    'Company Name,Symbol' (e.g., "GODREJ CONSUMER PRODUCTS,NSE:GODREJCP-EQ").
    The agent must then parse this list to decide which symbol(s) to use.
    """
    print(f"[Tool Call] search_for_company_matches: Searching for '{company_query}', k={top_k}")
    try:
        docs = vectorstore.similarity_search(company_query, k=top_k)
        if not docs:
            return ["Error: No companies found matching that query."]
        results = [doc.page_content for doc in docs]
        print(f"[Tool Result] Found matches: {results}")
        return results
    except Exception as e:
        print(f"[Tool Error] {e}")
        return [f"Error during company search: {e}"]

@tool
def get_current_prices(symbols: list[str]) -> dict:
    """
    Fetches the current last traded price (LTP) for a list of one or more
    valid trading symbols (e.g., ["NSE:RELIANCE-EQ", "NSE:TCS-EQ"]).
    Returns a dictionary where keys are symbols and values are their prices.
    """
    print(f"[Tool Call] get_current_prices: Fetching for {symbols}")
    if not symbols:
        return {"Error": "No symbols provided."}
    
    symbol_string = ",".join(symbols)
    data = {"symbols": symbol_string}
    
    try:
        response = fyers.quotes(data)
        if response.get("s") != "ok":
            print(f"[Tool Error] Fyers API error: {response.get('message')}")
            return {"Error": f"Fyers API error: {response.get('message')}"}
        
        price_results = {}
        if "d" in response and response["d"]:
            for item in response["d"]:
                symbol_name = item.get("n")
                last_price = item.get("v", {}).get("lp")
                if symbol_name and last_price is not None:
                    price_results[symbol_name] = last_price
            print(f"[Tool Result] Prices: {price_results}")
            return price_results
        else:
            print("[Tool Error] No 'd' key in Fyers response.")
            return {"Error": "No price data returned from Fyers."}
    except Exception as e:
        print(f"[Tool Error] {e}")
        return {"Error": f"Exception while calling Fyers API: {e}"}

# List of tools for the agent
tools = [search_for_company_matches, get_current_prices]
print("Tools defined.")

# -----------------------------
# Agent System Prompt
# -----------------------------
# This prompt now includes placeholders for the user profile and chat history
system_prompt_template = """You are a helpful finance assistant.
Your goal is to be as helpful as possible to the user.

Here is some information about the user you are talking to:
{user_profile}

You have access to tools that can find company trading symbols and get their current stock prices.
You must follow these steps:
1.  If the user mentions company names (e.g., "Reliance"), use `search_for_company_matches` to find potential matches.
2.  This tool returns a list (e.g., ["GODREJ CONSUMER PRODUCTS,NSE:GODREJCP-EQ", ...]).
3.  Analyze this list and the user's query. If one match is a clear fit, extract its symbol. If ambiguous, extract symbols for all relevant matches.
4.  Once you have symbol(s), use `get_current_prices` to fetch their price(s).
5.  Use the prices to answer the user's question.
6.  Always state the full company name and the symbol with the prices.

You have a conversation history. Use it to maintain context and avoid repeating questions.
"""

# -----------------------------
# Helper Function to List Threads
# -----------------------------
def list_threads(db_file):
    """Lists all unique session_ids from the SQLite database."""
    if not os.path.exists(db_file):
        return []
    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        # Check if table exists first
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='message_store';")
        if not cursor.fetchone():
            conn.close()
            return []
        
        cursor.execute("SELECT DISTINCT session_id FROM message_store")
        threads = [row[0] for row in cursor.fetchall()]
        conn.close()
        return threads
    except Exception as e:
        print(f"Error reading thread list from SQLite: {e}")
        return []

# -----------------------------
# Main Chat Loop (with Session Management)
# -----------------------------
print("\nFinance ChatBot Agent is ready!")

# Outer loop for session management
while True:
    session_id = None
    print("\n" + "="*30)
    print("     SESSION MANAGEMENT")
    print("="*30)
    choice = input("Start a (N)ew thread or load an (O)ld one? (N/O): ").upper()
    
    if choice == 'O':
        threads = list_threads(DB_FILE)
        if not threads:
            print("No old threads found. Starting a new one.")
            session_id = str(uuid.uuid4())
        else:
            print("\nAvailable threads:")
            for i, thread in enumerate(threads):
                print(f"  {i+1}: {thread}")
            try:
                thread_choice = int(input("Enter the number of the thread to load: "))
                if 1 <= thread_choice <= len(threads):
                    session_id = threads[thread_choice - 1]
                else:
                    raise ValueError("Choice out of range")
            except (ValueError, IndexError):
                print("Invalid choice. Starting a new thread.")
                session_id = str(uuid.uuid4())
    
    elif choice == 'N':
        session_id = str(uuid.uuid4())
        print(f"Starting new thread: {session_id}")
    
    else:
        print("Invalid choice. Please type 'N' or 'O'.")
        continue # Restart session loop

    print(f"\n[Session Active] ID: {session_id}")

    # --- 1. Set up session-specific memory ---
    # This links the memory to our SQLite DB using the session_id
    chat_history_backend = SQLChatMessageHistory(
        session_id=session_id,
        connection_string=SQL_CONNECTION_STRING
    )
    
    # This memory summarizes when tokens > 1000, and keeps recent
    # messages in a buffer.
    memory = ConversationSummaryBufferMemory(
        llm=llm, # Use the llm instance for summarization
        chat_memory=chat_history_backend, # Backend to save/load from
        max_token_limit=1000, # Summarizes messages beyond this token count
        memory_key="chat_history", # Must match placeholder in prompt
        return_messages=True
    )
    
    # --- 2. Create session-specific agent ---
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt_template.format(user_profile=USER_PROFILE)),
            ("placeholder", "{chat_history}"), # Key for memory
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"), # For agent's internal steps
        ]
    )

    agent = create_tool_calling_agent(llm, tools, prompt)

    # The executor ties the agent, tools, and memory together
    agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools, 
        memory=memory, # Pass the session-specific memory
        verbose=True
    )

    # --- 3. Start the chat loop for this session ---
    print(f"Chat ready! (Session: {session_id}). Type 'exit' to end session.")
    while True:
        user_query = input("You: ")
        if user_query.lower() in ["exit", "quit"]:
            print(f"[Session Ended] ID: {session_id}")
            break # Break inner loop, go back to session menu

        try:
            # The executor will:
            # 1. Load history from memory (which pulls from SQLite)
            # 2. Invoke the agent
            # 3. Save the new turn to memory (which writes to SQLite)
            response = agent_executor.invoke({"input": user_query})
            print("Bot:", response['output'])
        except Exception as e:
            print(f"An error occurred: {e}")

Environment variables loaded.
Company symbol vector store loaded!
LLM Initialized (llama3-70b-8192).
Fyers connection successful.
Tools defined.

Finance ChatBot Agent is ready!

     SESSION MANAGEMENT
Invalid choice. Please type 'N' or 'O'.

     SESSION MANAGEMENT
Starting new thread: f1b31d8e-1d16-4724-9709-8af479824cf0

[Session Active] ID: f1b31d8e-1d16-4724-9709-8af479824cf0
Chat ready! (Session: f1b31d8e-1d16-4724-9709-8af479824cf0). Type 'exit' to end session.


C:\Users\Asus\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Asus\AppData\Local\Temp\ipykernel_15024\2794146787.py:250: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(




> Entering new AgentExecutor chain...
Hello Rahul! How can I assist you today with your FinTech, reinforcement learning, or AI‑powered trading interests?


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)



> Finished chain.
Bot: Hello Rahul! How can I assist you today with your FinTech, reinforcement learning, or AI‑powered trading interests?


> Entering new AgentExecutor chain...

Invoking: `search_for_company_matches` with `{'company_query': 'TCS', 'top_k': 3}`


[Tool Call] search_for_company_matches: Searching for 'TCS', k=3
[Tool Result] Found matches: ['Company: TATA CONSULTANCY SERV LT, Symbol: NSE:TCS-EQ', 'Company: TCI EXPRESS LIMITED, Symbol: NSE:TCIEXP-EQ', 'Company: TCPL PACKAGING LIMITED, Symbol: NSE:TCPLPACK-EQ']
['Company: TATA CONSULTANCY SERV LT, Symbol: NSE:TCS-EQ', 'Company: TCI EXPRESS LIMITED, Symbol: NSE:TCIEXP-EQ', 'Company: TCPL PACKAGING LIMITED, Symbol: NSE:TCPLPACK-EQ']
Invoking: `search_for_company_matches` with `{'company_query': 'Tata Motors', 'top_k': 3}`


[Tool Call] search_for_company_matches: Searching for 'Tata Motors', k=3
[Tool Result] Found matches: ['Company: TATA MOTORS LIMITED, Symbol: NSE:TATAMOTORS-EQ', 'Company: TATA STEEL LIMITED, Symbol: N

## F&O

In [ ]:
import os
import re
import uuid  # For generating unique thread IDs
import sqlite3 # For listing existing threads
from dotenv import load_dotenv
import pandas as pd # For loading CSVs to create vector stores
import sys # <-- IMPORT SYS MODULE

# --- Core LangChain/Groq Imports ---
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain.docstore.document import Document # To create documents for FAISS

# --- New Imports for Memory ---
from langchain.memory import ConversationSummaryBufferMemory
from langchain_community.chat_message_histories import SQLChatMessageHistory # Backend for memory

# --- Fyers API ---
from fyers_apiv3 import fyersModel

# -----------------------------
# Configuration
# -----------------------------
DB_FILE = "chat_history.db" # File to store chat history
SQL_CONNECTION_STRING = f"sqlite:///{DB_FILE}"

# Vector Store Configuration
EQUITY_CSV_FILE = "symbols.csv"  # Assuming your original file was named this
EQUITY_FAISS_INDEX = "faiss_index"

FNO_CSV_FILE = "F&O_symbols.csv" # The new F&O symbols file
FNO_FAISS_INDEX = "fno_faiss_index" # The new index for F&O symbols

# As requested: Manually added user info for customization
USER_PROFILE = """
- User Name: [Rahul]
- Interests: FinTech, Reinforcement Learning, AI-powered trading
- Preferred Response: Factual, concise, and provide strategies.
"""

# -----------------------------
# Load environment variables
# -----------------------------
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
# CLIENT_ID = os.getenv("FYERS_CLIENT_ID")
# access_token = os.getenv("FYERS_TOKEN")

if not GROQ_API_KEY or not CLIENT_ID or not access_token:
    print("Error: Missing environment variables.")
    print("Please set GROQ_API_KEY, FYERS_CLIENT_ID, and FYERS_TOKEN in your .env file.")
    sys.stdout.flush() # <-- ADD FLUSH
    exit()

print("Environment variables loaded.")
sys.stdout.flush() # <-- ADD FLUSH

# -----------------------------
# Helper: Create Vector Store
# -----------------------------
def create_vector_store_if_missing(csv_path: str, index_path: str, embeddings_model):
    """Creates and saves a FAISS vector store if it doesn't already exist."""
    if os.path.exists(index_path):
        print(f"Vector store '{index_path}' already exists. Loading...")
        sys.stdout.flush() # <-- ADD FLUSH
        return
    
    print(f"Vector store '{index_path}' not found. Creating from '{csv_path}'...")
    sys.stdout.flush() # <-- ADD FLUSH
    if not os.path.exists(csv_path):
        print(f"Error: CSV file not found at '{csv_path}'. Cannot create vector store.")
        sys.stdout.flush() # <-- ADD FLUSH
        exit()
        
    try:
        df = pd.read_csv(csv_path)
        # Ensure 'Company name' and 'Symbol' columns exist
        if 'Company name' not in df.columns or 'Symbol' not in df.columns:
            print(f"Error: CSV '{csv_path}' must contain 'Company name' and 'Symbol' columns.")
            sys.stdout.flush() # <-- ADD FLUSH
            exit()
            
        # Create LangChain documents
        # The page_content is what the agent will see.
        documents = [
            Document(page_content=f"{row['Company name']},{row['Symbol']}")
            for _, row in df.iterrows()
        ]
        
        if not documents:
            print(f"Error: No documents created from '{csv_path}'. Is the file empty?")
            sys.stdout.flush() # <-- ADD FLUSH
            exit()
            
        # Create and save the vector store
        vectorstore = FAISS.from_documents(documents, embeddings_model)
        vectorstore.save_local(index_path)
        print(f"Successfully created and saved vector store at '{index_path}'.")
        sys.stdout.flush() # <-- ADD FLUSH
        
    except Exception as e:
        print(f"Error creating vector store from '{csv_path}': {e}")
        sys.stdout.flush() # <-- ADD FLUSH
        exit()

# -----------------------------
# Load Vector Stores
# -----------------------------
try:
    embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    # --- Load Equity Vector Store ---
    if not os.path.exists(EQUITY_FAISS_INDEX):
        print(f"Error: Equity vector store '{EQUITY_FAISS_INDEX}' not found.")
        print(f"Please create it from your equity CSV first.")
        sys.stdout.flush() # <-- ADD FLUSH
        # As a fallback, I'll try to create it from EQUITY_CSV_FILE
        print(f"Attempting to create '{EQUITY_FAISS_INDEX}' from '{EQUITY_CSV_FILE}'...")
        sys.stdout.flush() # <-- ADD FLUSH
        create_vector_store_if_missing(EQUITY_CSV_FILE, EQUITY_FAISS_INDEX, embeddings_model)
        
    equity_vectorstore = FAISS.load_local(EQUITY_FAISS_INDEX, embeddings_model, allow_dangerous_deserialization=True)
    print("Equity symbol vector store loaded!")
    sys.stdout.flush() # <-- ADD FLUSH

    # --- Create/Load F&O Vector Store ---
    create_vector_store_if_missing(FNO_CSV_FILE, FNO_FAISS_INDEX, embeddings_model)
    fno_vectorstore = FAISS.load_local(FNO_FAISS_INDEX, embeddings_model, allow_dangerous_deserialization=True)
    print("F&O symbol vector store loaded!")
    sys.stdout.flush() # <-- ADD FLUSH

except Exception as e:
    print(f"Error loading vector stores: {e}")
    sys.stdout.flush() # <-- ADD FLUSH
    exit()

# -----------------------------
# Initialize LLM
# -----------------------------
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model_name="openai/gpt-oss-120b", # Using a larger, more capable model
    temperature=0.1
)
print("LLM Initialized (llama3-70b-8192).")
sys.stdout.flush() # <-- ADD FLUSH

# -----------------------------
# Initialize Fyers model
# -----------------------------
try:
    fyers = fyersModel.FyersModel(client_id=CLIENT_ID, token=access_token, is_async=False)
    test_response = fyers.get_profile()
    if test_response.get('s') != 'ok':
        print(f"Fyers API Error: {test_response.get('message')}")
        sys.stdout.flush() # <-- ADD FLUSH
        exit()
    print("Fyers connection successful.")
    sys.stdout.flush() # <-- ADD FLUSH
except Exception as e:
    print(f"Error initializing Fyers model: {e}")
    sys.stdout.flush() # <-- ADD FLUSH
    exit()

# -----------------------------
# Define Tools
# -----------------------------

@tool
def search_for_equity_symbol(company_query: str, top_k: int = 3) -> list[str]:
    """
    Searches the EQUITY vector database for the top_k (default 3) most similar
    company names for a given query.
    Returns a list of raw strings, where each string contains the
    'Company Name,Symbol' (e.g., "TATA CONSULTANCY SERV LT,NSE:TCS-EQ").
    This tool should be used to find the UNDERLYING EQUITY symbol for stocks
    and for getting options data.
    """
    print(f"[Tool Call] search_for_equity_symbol: Searching for '{company_query}', k={top_k}")
    sys.stdout.flush() # <-- ADD FLUSH
    try:
        docs = equity_vectorstore.similarity_search(company_query, k=top_k)
        if not docs:
            return ["Error: No equity symbols found matching that query."]
        results = [doc.page_content for doc in docs]
        print(f"[Tool Result] Found matches: {results}")
        sys.stdout.flush() # <-- ADD FLUSH
        return results
    except Exception as e:
        print(f"[Tool Error] {e}")
        sys.stdout.flush() # <-- ADD FLUSH
        return [f"Error during equity search: {e}"]

@tool
def search_for_fno_symbol(derivative_query: str, top_k: int = 3) -> list[str]:
    """
    Searches the F&O (Futures & Options) vector database for the top_k (default 3)
    most similar derivative contracts, like futures.
    Returns a list of raw strings, where each string contains the
    'Contract Name,Symbol' (e.g., "TCS 28OCT2025 FUT,NSE:TCS25OCTFUT").
    This tool should be used to find specific FUTURES contracts.
    """
    print(f"[Tool Call] search_for_fno_symbol: Searching for '{derivative_query}', k={top_k}")
    sys.stdout.flush() # <-- ADD FLUSH
    try:
        docs = fno_vectorstore.similarity_search(derivative_query, k=top_k)
        if not docs:
            return ["Error: No F&O symbols found matching that query."]
        results = [doc.page_content for doc in docs]
        print(f"[Tool Result] Found matches: {results}")
        sys.stdout.flush() # <-- ADD FLUSH
        return results
    except Exception as e:
        print(f"[Tool Error] {e}")
        sys.stdout.flush() # <-- ADD FLUSH
        return [f"Error during F&O search: {e}"]

@tool
def get_current_prices(symbols: list[str]) -> dict:
    """
    Fetches the current last traded price (LTP) for a list of one or more
    valid trading symbols (e.g., ["NSE:RELIANCE-EQ", "NSE:TCS25OCTFUT"]).
    This works for both equities and futures.
    Returns a dictionary where keys are symbols and values are their prices.
    """
    print(f"[Tool Call] get_current_prices: Fetching for {symbols}")
    sys.stdout.flush() # <-- ADD FLUSH
    if not symbols:
        return {"Error": "No symbols provided."}
    
    symbol_string = ",".join(symbols)
    data = {"symbols": symbol_string}
    
    try:
        response = fyers.quotes(data)
        if response.get("s") != "ok":
            print(f"[Tool Error] Fyers API error: {response.get('message')}")
            sys.stdout.flush() # <-- ADD FLUSH
            return {"Error": f"Fyers API error: {response.get('message')}"}
        
        price_results = {}
        if "d" in response and response["d"]:
            for item in response["d"]:
                symbol_name = item.get("n")
                last_price = item.get("v", {}).get("lp")
                if symbol_name and last_price is not None:
                    price_results[symbol_name] = last_price
            print(f"[Tool Result] Prices: {price_results}")
            sys.stdout.flush() # <-- ADD FLUSH
            return price_results
        else:
            print("[Tool Error] No 'd' key in Fyers response.")
            sys.stdout.flush() # <-- ADD FLUSH
            return {"Error": "No price data returned from Fyers."}
    except Exception as e:
        print(f"[Tool Error] {e}")
        sys.stdout.flush() # <-- ADD FLUSH
        return {"Error": f"Exception while calling Fyers API: {e}"}

@tool
def get_available_expiries(underlying_symbol: str) -> dict:
    """
    Fetches all available option expiry dates for a given UNDERLYING EQUITY symbol
    (e.g., "NSE:TCS-EQ").
    Returns a dictionary containing a list of expiry data.
    """
    print(f"[Tool Call] get_available_expiries: Fetching for {underlying_symbol}")
    sys.stdout.flush() # <-- ADD FLUSH
    data = {"symbol": underlying_symbol}
    try:
        response = fyers.optionchain(data=data)
        if response.get("s") != "ok":
            print(f"[Tool Error] Fyers API error: {response.get('message')}")
            sys.stdout.flush() # <-- ADD FLUSH
            return {"Error": f"Fyers API error: {response.get('message')}"}
        
        expiry_data = response.get("data", {}).get("expiryData", [])
        if not expiry_data:
            return {"Error": "No expiry data found for this symbol."}
            
        print(f"[Tool Result] Found {len(expiry_data)} expiries.")
        sys.stdout.flush() # <-- ADD FLUSH
        # Return the 'expiryData' list directly as it's serializable
        return {"expiryData": expiry_data}
        
    except Exception as e:
        print(f"[Tool Error] {e}")
        sys.stdout.flush() # <-- ADD FLUSH
        return {"Error": f"Exception while calling Fyers API: {e}"}

@tool
def get_option_chain_data(underlying_symbol: str, expiry_timestamp: str) -> dict:
    """
    Fetches the complete option chain for a given UNDERLYING EQUITY symbol
    (e.g., "NSE:TCS-EQ") and a specific expiry timestamp (e.g., "1761645600").
    The timestamp MUST be one of the 'expiry' values from 'get_available_expiries'.
    Returns a dictionary with the spot price and a list of tuples:
    (strike_price, premium_ltp, option_type 'CE'/'PE').
    """
    print(f"[Tool Call] get_option_chain_data: Fetching for {underlying_symbol} at {expiry_timestamp}")
    sys.stdout.flush() # <-- ADD FLUSH
    data = {"symbol": underlying_symbol, "timestamp": expiry_timestamp}
    try:
        response = fyers.optionchain(data=data)
        if response.get("s") != "ok":
            print(f"[Tool Error] Fyers API error: {response.get('message')}")
            sys.stdout.flush() # <-- ADD FLUSH
            return {"Error": f"Fyers API error: {response.get('message')}"}

        options_chain_list = response.get("data", {}).get("optionsChain", [])
        if not options_chain_list:
            return {"Error": "No option chain data found for this symbol and expiry."}

        # The first item [0] is the underlying equity data (spot price)
        spot_price = options_chain_list[0].get("ltp")

        # The rest of the items [1:] are the options
        options_list = []
        for item in options_chain_list[1:]:
            strike = item.get("strike_price")
            ltp = item.get("ltp")
            opt_type = item.get("option_type")
            if strike is not None and ltp is not None and opt_type in ('CE', 'PE'):
                options_list.append((strike, ltp, opt_type))
        
        result = {
            "spot_price": spot_price,
            "options": options_list
        }
        print(f"[Tool Result] Found spot price {spot_price} and {len(options_list)} options.")
        sys.stdout.flush() # <-- ADD FLUSH
        return result

    except Exception as e:
        print(f"[Tool Error] {e}")
        sys.stdout.flush() # <-- ADD FLUSH
        return {"Error": f"Exception while calling Fyers API: {e}"}


# List of ALL tools for the agent
tools = [
    search_for_equity_symbol, 
    search_for_fno_symbol,
    get_current_prices,
    get_available_expiries,
    get_option_chain_data
]
print("Tools defined.")
sys.stdout.flush() # <-- ADD FLUSH

# -----------------------------
# Agent System Prompt
# -----------------------------
# *** UPDATED PROMPT with corrected logic and KeyError fix ***
system_prompt_template = """You are a helpful finance assistant.
Your goal is to be as helpful as possible to the user.

Here is some information about the user you are talking to:
{user_profile}

You have access to tools to get prices for equities, futures, and options.
You MUST follow these workflows and logic:

**CRITICAL RULE:** When searching for a company, search for the COMPANY NAME ONLY.
-   If the user asks for "TCS futures price", search for "TCS" or "Tata Consultancy".
-   If the user asks for "Reliance stock price", search for "Reliance".
-   DO NOT include "futures", "options", "stock", "PE", or "CE" in your search queries.

**Workflow 1: Get Equity (Stock) Price**
1.  User asks for a stock price (e.g., "What is the price of Reliance?").
2.  Use `search_for_equity_symbol` with the company name (e.g., "Reliance").
3.  Extract the equity symbol (e.g., "NSE:RELIANCE-EQ") from the results.
4.  Use `get_current_prices` with the found symbol(s) to get the LTP.
5.  Report the full company name, symbol, and price.

**Workflow 2: Get Futures Price**
1.  User asks for a futures price (e.g., "TCS futures price?").
2.  Use `search_for_fno_symbol` with the company name (e.g., "TCS").
3.  This tool will return matches like "TCS 28OCT2025 FUT,NSE:TCS25OCTFUT". Pick the most relevant one, usually the nearest expiry.
4.  Use `get_current_prices` with the found futures symbol (e.g., ["NSE:TCS25OCTFUT"]) to get its LTP.
5.  Report the contract name, symbol, and price.

**Workflow 3: Get Options Data or Expiries**
1.  User asks for options data (e.g., "Show me TCS options" or "What are the expiries for TCS?").
2.  FIRST, use `search_for_equity_symbol` with the company name (e.g., "TCS") to get the UNDERLYING EQUITY symbol (e.g., "NSE:TCS-EQ"). All options tools require this base symbol.
3.  NEXT, use `get_available_expiries` with the underlying symbol ("NSE:TCS-EQ").
4.  This will return a list of expiry dates (e.g., [date: '28-10-2025', expiry: '1761645600', ...]).
5.  **If the user *only* asked for expiries**, present this list of dates.
6.  **If the user asked for options data (e.g., "TCS options"):**
    a.  Present the list of available expiry dates and ASK the user which one they want.
    b.  If the user doesn't specify, **default to the FIRST expiry** in the list.
    c.  Get the 'expiry' timestamp (e.g., "1761645600") for the chosen date.
    d.  Call `get_option_chain_data` with the underlying symbol ("NSE:TCS-EQ") and the chosen timestamp ("1761645600").
    e.  This returns a spot price and a LONG list of options.
    f.  Report the spot price and a *summary* of the options. For example: "The spot price is 2962.2. I found 200 options for the 28-10-2025 expiry. Strikes near the spot price include..." DO NOT print the full list unless asked.

You have a conversation history. Use it to maintain context (e.g., if you just listed expiries, you know the underlying symbol).
"""
# -----------------------------
# Helper Function to List Threads
# -----------------------------
def list_threads(db_file):
    """Lists all unique session_ids from the SQLite database."""
    if not os.path.exists(db_file):
        return []
    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        # Check if table exists first
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='message_store';")
        if not cursor.fetchone():
            conn.close()
            return []
        
        cursor.execute("SELECT DISTINCT session_id FROM message_store")
        threads = [row[0] for row in cursor.fetchall()]
        conn.close()
        return threads
    except Exception as e:
        print(f"Error reading thread list from SQLite: {e}")
        sys.stdout.flush() # <-- ADD FLUSH
        return []

# -----------------------------
# Main Chat Loop (with Session Management)
# -----------------------------
print("\nFinance ChatBot Agent is ready!")
sys.stdout.flush() # <-- ADD FLUSH

# Outer loop for session management
while True:
    session_id = None
    print("\n" + "="*30)
    print("     SESSION MANAGEMENT")
    print("="*30)
    sys.stdout.flush() # <-- ADD FLUSH
    choice = input("Start a (N)ew thread or load an (O)ld one? (N/O): ").upper()
    
    if choice == 'O':
        threads = list_threads(DB_FILE)
        if not threads:
            print("No old threads found. Starting a new one.")
            sys.stdout.flush() # <-- ADD FLUSH
            session_id = str(uuid.uuid4())
        else:
            print("\nAvailable threads:")
            sys.stdout.flush() # <-- ADD FLUSH
            for i, thread in enumerate(threads):
                print(f"  {i+1}: {thread}")
                sys.stdout.flush() # <-- ADD FLUSH
            try:
                thread_choice = int(input("Enter the number of the thread to load: "))
                if 1 <= thread_choice <= len(threads):
                    session_id = threads[thread_choice - 1]
                else:
                    raise ValueError("Choice out of range")
            except (ValueError, IndexError):
                print("Invalid choice. Starting a new thread.")
                sys.stdout.flush() # <-- ADD FLUSH
                session_id = str(uuid.uuid4())
    
    elif choice == 'N':
        session_id = str(uuid.uuid4())
        print(f"Starting new thread: {session_id}")
        sys.stdout.flush() # <-- ADD FLUSH
    
    else:
        print("Invalid choice. Please type 'N' or 'O'.")
        sys.stdout.flush() # <-- ADD FLUSH
        continue # Restart session loop

    print(f"\n[Session Active] ID: {session_id}")
    sys.stdout.flush() # <-- ADD FLUSH

    # --- 1. Set up session-specific memory ---
    chat_history_backend = SQLChatMessageHistory(
        session_id=session_id,
        connection_string=SQL_CONNECTION_STRING
    )
    
    memory = ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=chat_history_backend,
        max_token_limit=1000,
        memory_key="chat_history",
        return_messages=True
    )
    
    # --- 2. Create session-specific agent ---
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt_template.format(user_profile=USER_PROFILE)),
            ("placeholder", "{chat_history}"), # Key for memory
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"), # For agent's internal steps
        ]
    )

    agent = create_tool_calling_agent(llm, tools, prompt)

    agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools, 
        memory=memory,
        verbose=True
    )

    # --- 3. Start the chat loop for this session ---
    print(f"Chat ready! (Session: {session_id}). Type 'exit' to end session.")
    sys.stdout.flush() # <-- ADD FLUSH
    while True:
        user_query = input("You: ")
        print("\nYOU :: ", user_query)
        
        # Don't print the user's query again, just process it.
        # print("\nYOU :: ", user_query) # <-- This was redundant
        
        if user_query.lower() in ["exit", "quit"]:
            print(f"[Session Ended] ID: {session_id}")
            sys.stdout.flush() # <-- ADD FLUSH
            break # Break inner loop, go back to session menu
        
        # Check for empty input and re-prompt if necessary
        if not user_query.strip():
            print("Please enter a query.")
            sys.stdout.flush()
            continue

        try:
            response = agent_executor.invoke({"input": user_query})
            print("Bot:", response['output'])
            sys.stdout.flush() # <-- ADD FLUSH
        except Exception as e:
            print(f"An error occurred: {e}")
            sys.stdout.flush() # <-- ADD FLUSH

Environment variables loaded.
Equity symbol vector store loaded!
Vector store 'fno_faiss_index' already exists. Loading...
F&O symbol vector store loaded!
LLM Initialized (llama3-70b-8192).
Fyers connection successful.
Tools defined.

Finance ChatBot Agent is ready!

     SESSION MANAGEMENT
Starting new thread: bc21d6ae-8677-4627-8c6e-8f46f7a3fecc

[Session Active] ID: bc21d6ae-8677-4627-8c6e-8f46f7a3fecc
Chat ready! (Session: bc21d6ae-8677-4627-8c6e-8f46f7a3fecc). Type 'exit' to end session.

YOU ::  
Please enter a query.

YOU ::  Who are you


> Entering new AgentExecutor chain...
I’m a finance‑focused virtual assistant. I can look up real‑time equity, futures and options prices, list option expiries, and give concise market summaries—especially useful if you’re interested in FinTech, reinforcement learning or AI‑powered trading. Let me know what data you need!

> Finished chain.
Bot: I’m a finance‑focused virtual assistant. I can look up real‑time equity, futures and options prices